# Setup of the GOTM/FABM model for Langtjern

## Compiling GOTM and FABM

Please follow the instructions at <https://github.com/gotm-model/code>. Please note that the provided instructions are only available for Linux systems at present. 

For Windows, a combination of Visual Studio, CMake and the Intel Fortran compiler should work with minimal modifications when CMake is setup to generate Visual Studio project files.

For the rest of this document, it is assumed that Linux is used in combination with CMake, using GCC-5.4. 